In [ ]:
!pip install ddgs

In [ ]:
import os
import requests
from bs4 import BeautifulSoup

# Install langchain-community if it's not already installed
try:
    from langchain_community.vectorstores import Chroma
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.tools import DuckDuckGoSearchRun
except ImportError:
    !pip install -U langchain-community
    from langchain_community.vectorstores import Chroma
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.tools import DuckDuckGoSearchRun

# Install chromadb if it's not already installed
try:
    import chromadb
except ImportError:
    !pip install chromadb
    import chromadb

from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_groq import ChatGroq

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "XX"


Initializing the tools/modeules that needs to used


In [ ]:

DB_DIR = os.path.join(os.getcwd(), "university_db")  # Directory/folder name in chromadb where it stores the information

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectordb = Chroma(
    collection_name="university_assistant",
    embedding_function=embedding,
    persist_directory=DB_DIR,
)

retriever = vectordb.as_retriever(search_kwargs={"k": 4})


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Total web search and db storing component


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)


def scrape_website(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0 (compatible; UniversityAssistant/1.0)"}
        response = requests.get(url, timeout=10, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(separator=" ", strip=True)
        return text
    except Exception:
        return None

UNIVERSITY_SITE = "uta.edu"  # change this
search_tool = DuckDuckGoSearchRun(num_results=6)


def search_university_site(question):
    query = f"site:{UNIVERSITY_SITE} {question}"
    return search_tool.run(query)


def extract_valid_urls(search_results):
    urls = []

    if isinstance(search_results, str):
        import re

        urls.extend(re.findall(r"(https?://\S+)", search_results))
        return urls

    for entry in search_results or []:
        link = entry.get("link") or entry.get("href")
        if link and link.startswith("http"):
            urls.append(link)

    return urls



def store_web_content(text, source):
    docs = [Document(page_content=text, metadata={"source": source})]
    splits = text_splitter.split_documents(docs)
    vectordb.add_documents(splits)
    vectordb.persist()



In [ ]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a university assistant at University of Texas at Arlington(UTA). Answer ONLY from the provided context. "
        "If the answer is not in context, use web search options.",
    ),
    ("human", "Question: {question}\n\nContext:\n{context}"),
])

qa_chain = prompt | llm | StrOutputParser()


def format_docs(docs):
    if not docs:
        return ""
    return "\n\n".join(d.page_content for d in docs)


def needs_web_search(answer):
    normalized = answer.strip().lower()
    return normalized in {"not_found", "not found", "not_found."} or not normalized


def answer_from_context(question):
    docs = retriever.invoke(question)
    context = format_docs(docs)
    answer = qa_chain.invoke({"question": question, "context": context})
    return answer.strip(), docs


In [ ]:
def university_assistant(question, max_urls=3):
    answer, _ = answer_from_context(question)

    if not needs_web_search(answer):
        return answer

    print("🔎 Searching university website...")
    search_results = search_university_site(question)
    candidate_urls = extract_valid_urls(search_results)

    if not candidate_urls:
        return "Sorry, I couldn't find any relevant university pages to check."

    ingested = 0
    for url in candidate_urls[:max_urls]:
        web_text = scrape_website(url)
        if not web_text:
            continue
        store_web_content(web_text, url)
        ingested += 1

    if ingested == 0:
        return "I checked the university site but could not ingest any usable content."

    print(f"📚 Stored {ingested} new page(s) into the vector DB. Re-running the answer...")
    refreshed_answer, _ = answer_from_context(question)

    if needs_web_search(refreshed_answer):
        return "I searched the university site but still cannot find a reliable answer."

    return refreshed_answer


In [ ]:
def chat():
    print("🎓 University Assistant is ready!")
    print("Type 'exit' or 'quit' to stop.\n")

    prompt_text = "Type your question here: "

    while True:
        try:
            q = input(prompt_text).strip()

            if not q:
                continue

            if q.lower() in ["exit", "quit"]:
                print("👋 Goodbye!")
                break

            response = university_assistant(q)
            print("\nAssistant:", response, "\n")

        except KeyboardInterrupt:
            print("\n👋 Goodbye!")
            break
        except Exception as e:
            print("⚠️ Error:", str(e))


In [ ]:
chat()


🎓 University Assistant is ready!
Type 'exit' or 'quit' to stop.

Type your question here: When was UTA Established?

Assistant: The University of Texas at Arlington (UTA) was established in 1895 as the Arlington College. 

Type your question here: How far is UTA from Dallas?

Assistant: The University of Texas at Arlington (UTA) is located in Arlington, Texas. 

According to Google Maps, the driving distance from UTA to Dallas is approximately 20 miles (32 kilometers). The drive typically takes around 30-40 minutes, depending on traffic conditions. 


👋 Goodbye!
